In [ ]:
from ultralytics import YOLO
import skimage as ski
from sklearn.metrics import accuracy_score
from PIL import Image, ImageDraw
from glob import glob
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

In [ ]:
def convert(labels, width, height):
    '''
    Convert the label to the points    
    '''
    lines_sp = labels.split()
    class_index = int(lines_sp[0])
    coordinates = list(lines_sp[1:])

    x_coords = coordinates[::2]
    y_coords = coordinates[1::2]

    normalized_x_coords = [float(coord) * width for coord in x_coords]
    normalized_y_coords = [float(coord) * height for coord in y_coords]

    points = list(zip(normalized_x_coords, normalized_y_coords))
    
    return points, class_index


def get_length(masks):    
    """
    Find the length of the mask
    """
    rows, _ = np.where(masks > 0)
    row_center = rows.min() + int((rows.max() - rows.min()) / 2)
    cols = np.where(masks[row_center, :] > 0)[0]
    height = cols.max() - cols.min()
    
    return height


def calculate_iou(gt_mask, pred_mask):
    """
    Calculate the iou score
    """
    intersection = np.logical_and(gt_mask, pred_mask)
    union = np.logical_or(gt_mask, pred_mask)
    iou_score = np.sum(intersection) / np.sum(union)
    
    return iou_score


def calculate_dice(gt_mask, pred_mask):
    """
    Calculate the dice score
    """
    intersection = np.logical_and(gt_mask, pred_mask)
    dice_score = 2 * np.sum(intersection) / (np.sum(gt_mask) + np.sum(pred_mask))
    
    return dice_score

def miou_dice_score(df_visualize, check_num, disc_pred, cup_pred):
    """
    Calculate the miou score
    """
    image_path = df_visualize["img"][check_num]
    image = Image.open(image_path)

    label_file_path = df_visualize["label"][check_num]
    with open(label_file_path, 'r') as f:
        labels = f.readlines()

    cup_line, disc_lines = labels

    width, height = image.size
    cup_label, class_index_label_1 = convert(cup_line, width, height)
    disc_label, class_index_label_2 = convert(disc_lines, width, height)

    image_shape = image.size
    mask_pred_cup = ski.draw.polygon2mask(image_shape, cup_pred)
    mask_pred_disc = ski.draw.polygon2mask(image_shape, disc_pred)

    mask_label_cup = ski.draw.polygon2mask(image_shape, cup_label) # cup
    mask_label_disc = ski.draw.polygon2mask(image_shape, disc_label) # disc

    iou_cup = calculate_iou(mask_pred_cup, mask_label_cup)
    iou_disc = calculate_iou(mask_pred_disc, mask_label_disc)

    dice_cup = calculate_dice(mask_pred_cup, mask_label_cup)
    dice_disc = calculate_dice(mask_pred_disc, mask_label_disc)

    return iou_cup, iou_disc, dice_cup, dice_disc



def cdr_score(df_visualize, check_num, disc_pred, cup_pred):
    """
    Calculate the cdr score
    """
    image_path = df_visualize["img"][check_num]
    image = Image.open(image_path)

    label_file_path = df_visualize["label"][check_num]
    with open(label_file_path, 'r') as f:
        labels = f.readlines()
    cup_line, disc_lines = labels

    width, height = image.size
    cup_label, _ = convert(cup_line, width, height)
    disc_label, _ = convert(disc_lines, width, height)

    cup_predict = []
    for cup_pt in cup_pred:
        mask_cup = tuple(np.array(cup_pt, dtype=np.float32).tolist())
        cup_predict.append(mask_cup)
    
    disc_predict = []
    for disc_pt in disc_pred:
        mask_disc = tuple(np.array(disc_pt, dtype=np.float32).tolist())
        disc_predict.append(mask_disc)
    
    image_shape = image.size
    mask_pred_cup = ski.draw.polygon2mask(image_shape, cup_predict)
    mask_pred_disc = ski.draw.polygon2mask(image_shape, disc_predict)

    mask_label_cup = ski.draw.polygon2mask(image_shape, cup_label)
    mask_label_disc = ski.draw.polygon2mask(image_shape, disc_label)
    
    cup_pred_lg = get_length(mask_pred_cup)
    disc_pred_lg = get_length(mask_pred_disc)

    cup_label_lg = get_length(mask_label_cup)
    disc_label_lg = get_length(mask_label_disc)

    return cup_pred_lg/disc_pred_lg, cup_label_lg/disc_label_lg, mask_label_cup, mask_label_disc


def calculate_area()

In [ ]:
img = glob("SI/test_si/*.jpg") 
label = glob("SI/test_si/*.txt")
img.sort()
label.sort()

df_visualize = pd.DataFrame({"img": img, "label": label})

model = YOLO("runs/segment/yolov8x-seg-si/weights/best.pt")  # load a custom model

In [ ]:
image_path = df_visualize["img"][2]
image = Image.open(image_path)
results = model(image) 

In [ ]:
name_img = []
cup_iou_score = []
disc_iou_score = []
cup_dice_score = []
disc_dice_score = []
cdr_preds_score = []
cdr_labels_score = []

missing_count = 0
one_class = 0
dk = 0
for check_num in tqdm(range(len(df_visualize))):

    image_path = df_visualize["img"][check_num]
    image = Image.open(image_path)
    
    results = model(image)  
    predict = model.predict(image, save=True, imgsz=1024, iou=0.1, show_conf=False)
    
    masks_predict = results[0].masks  

    if results[0].masks == None: # no mask
        missing_count += 1
        continue
    
    predict_seg = masks_predict.xy
    if len(predict_seg) == 2 :
        disc_pred = predict_seg[0]
        cup_pred = predict_seg[1]

    elif len(predict_seg) == 3 :
        disc_pred = predict_seg[1]
        cup_pred = predict_seg[2]

    else:
        one_class += 1
        continue

    if len(disc_pred) < 3 or len(cup_pred) == 0: 
        dk += 1
        continue
    
    cup_iou, disc_iou, cup_dice, disc_dice = miou_dice_score(df_visualize, check_num, disc_pred, cup_pred)
    cdr_pred, cdr_label, c,d  = cdr_score(df_visualize, check_num, disc_pred, cup_pred)

    break
    name_img.append(image_path.split("/")[-1])
    cup_iou_score.append(cup_iou)
    disc_iou_score.append(disc_iou)
    cup_dice_score.append(cup_dice)
    disc_dice_score.append(disc_dice)
    cdr_preds_score.append(cdr_pred)
    cdr_labels_score.append(cdr_label)

In [ ]:
name_img = []

from matplotlib import pyplot as plt
from scipy.ndimage import rotate

missing_count = 0
one_class = 0
dk = 0
for check_num in tqdm(range(len(df_visualize))):

    image_path = df_visualize["img"][check_num]
    image = Image.open(image_path)
    
    results = model(image)  
    predict = model.predict(image, save=True, imgsz=1024, iou=0.1, show_conf=False)
    
    masks_predict = results[0].masks  

    if results[0].masks == None: # no mask
        missing_count += 1
        continue
    
    predict_seg = masks_predict.xy
    if len(predict_seg) == 2 :
        disc_pred = predict_seg[0]
        cup_pred = predict_seg[1]

    elif len(predict_seg) == 3 :
        disc_pred = predict_seg[1]
        cup_pred = predict_seg[2]

    else:
        one_class += 1
        continue

    if len(disc_pred) < 3 or len(cup_pred) == 0: 
        dk += 1
        continue
    
    cup_iou, disc_iou, cup_dice, disc_dice = miou_dice_score(df_visualize, check_num, disc_pred, cup_pred)
    cdr_pred, cdr_label, c,d  = cdr_score(df_visualize, check_num, disc_pred, cup_pred)

    height, width = c.shape

    fig, ax = plt.subplots()
    
    rotated_image = rotate(image, 90)
    flipped_image = np.flipud(rotated_image)
    ax.imshow(c, cmap='gray', extent=[0, width, height, 0])
    ax.imshow(d, cmap='gray', alpha=0.5, extent=[0, width, height, 0])
    ax.imshow(flipped_image, alpha=0.5, extent=[0, width, height, 0])
    ax.axis('off')
    name_img = image_path.split("/")[-1]
    plt.savefig(f"test/{name_img}", bbox_inches='tight', pad_inches=0)


In [ ]:
print("predict :", len(cup_iou_score), "// 1 class: ", one_class, "// no detect: ", missing_count, "// dk: ", dk )
print("cup_miou_score : ", np.average(cup_iou_score))
print("disc_miou_score : ", np.average(cup_dice_score))

print("cup_dice_score : ", np.average(cup_dice_score))
print("disc_dice_score : ", np.average(disc_dice_score))

In [ ]:
df_predict = pd.DataFrame({"img_name":name_img, "cdr_preds_score": cdr_preds_score, "cdr_labels_score": cdr_labels_score})

In [ ]:
df_predict.to_csv("cdr_score.csv", index=False)

cdr graph

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve, PrecisionRecallDisplay
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, auc

In [ ]:
cdr_df = df_predict[df_predict.cdr_preds_score < 1]

In [ ]:
plt.scatter(cdr_df.cdr_labels_score, cdr_df.cdr_preds_score, alpha=0.7)
plt.xlabel("CDR (actual)")
plt.ylabel("CDR (pred)")
plt.axhline(y = 0.6, color = 'r', linestyle = '--') 
plt.axvline(x = 0.7, color = 'r', linestyle = '--') 
plt.show()

In [ ]:
for cutoff in np.arange(0.5, 0.95, 0.05):
    y_pred = (cdr_df.cdr_preds_score >= cutoff).astype(int)
    y_true = (cdr_df.cdr_labels_score >= 0.7).astype(int)
    print(f"Cutoff = {cutoff}")
    # print(precision_recall_fscore_support(y_true, y_pred, average=None))
    
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=["negative", "positive"])
    disp.plot(cmap=plt.cm.Blues)